# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

### Not for Grading

## Problem Statement

The problem is to identify the Suggest book from the conversation with the user, using the Python Chatbot approach.

### It is recommended to watch the Demystifying Chatbots Video

In [ ]:
#@title Demystifying Chatbot Video
from IPython.display import HTML

HTML("""<video width="854" height="480" controls>
  <source src="https://cdn.talentsprint.com/talentsprint/archives/sc/aiml/aiml_action_workshop_part_3.mp4" type="video/mp4">
</video>
""")


### It is recommended to watch the chatbot walkthrough video before you start working on the Pre-Hackathon

In [ ]:
#@title Chatbot Walkthrough Video
from IPython.display import HTML

HTML("""<video width="854" height="480" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Walkthrough/b17_pre_hackathon_1_chatbot_walkthrough.mp4" type="video/mp4">
</video>
""")

# Alexa Chatbot

- Create intent, slots, utterances and update the lambda function for zodiac sign intent
- While testing the correct zodiac sign should be uttered by the alexa
- Refer the  Pre-Hackathon for Alexa ChatBot material to understand Alexa Chatbot architecture and the implementation

# Python Chatbot

* For Suggest book Intent, all the required utterances, slots, and params (JSON) files are provided for your reference. A CSV file is also provided to perform the action.

### Below is the code for updating the Python Chatbot

In [ ]:
#@title Run this cell to download the data
!wget -qq https://cdn.iiith.talentsprint.com/aiml/Hackathon_data/Chatbot_Hackathon.zip
!unzip -qq Chatbot_Hackathon.zip
print("Data downloaded successfully")

Data downloaded successfully


In [1]:
!unzip -qq chat_bot_python.zip

In [2]:
# Import Libraries
import json
import random
import os
import re
import datetime
import pandas as pd
import numpy as np

# Importing context and .py script files
from Context import *
from Intent import *

### Chatbot Architecture

Defining functions for Loading Intent, Collecting params, Checking actions, Getting Attributes, and Identifying Intents

In [3]:
def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                return random.choice(para.prompts), context
    return None, context

def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent as mentioned
    in the intent config file. Performs actions pertaining to current intent '''
    context = IntentComplete()
    if current_intent.action.endswith('()'):
        return eval(current_intent.action), context
    return current_intent.action, context

def getattributes(uinput,context,attributes, intent):
    '''This function marks the slots in user input, and updates
    the attributes dictionary'''
    uinput = " "+uinput.lower()+" "
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        files = os.listdir(path_slots)
        slots = {}
        for fil in files:
            if fil == ".ipynb_checkpoints":
                continue
            lines = open(path_slots+fil).readlines()
            for i, line in enumerate(lines):
                line = line.strip()
                if len(uinput.split(" "+line.lower()+" ")) > 1:
                    slots[line] = fil[:-4]
        for value, slot in slots.items():
            if intent != None and slot in " ".join([param.name for param in intent.params]):
                uinput = re.sub(value,r'$'+slot,uinput,flags=re.IGNORECASE)
                attributes[slot] = value
            else:
                uinput = re.sub(value,r'$'+slot,uinput,flags=re.IGNORECASE)
                attributes[slot] = value
        return attributes, uinput

def input_processor(user_input, context, attributes, intent):
    '''Update the attributes, abstract over the slots in user input'''
    attributes, cleaned_input = getattributes(user_input, context, attributes, intent)
    return attributes, cleaned_input

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    if (current_intent==None):
        return loadIntent(path_param,intentPredict(clean_input))
    else:
        #If current intent is not none, stick with the ongoing intent
        #return current_intent
        intent = loadIntent(path_param,intentPredict(clean_input))
        if current_intent != intent:
            for para in current_intent.params:
                if para.name in clean_input:
                    return current_intent
        return loadIntent(path_param,intentPredict(clean_input))

Session class is one active session of the chatbot with which the user interacts. Let's go into the details:

**reply( )** is the important one in our session object it takes user_input as a parameter and calls different modules of the chatbot architecture:


*   **input_processor( )** - It helps in preprocessing and fetching the slots that can identify in the ready state
    
    - **getattributes( )** - It helps in identifying all the slots in the user utterance. Identify and map them to the parameters
    
    
*   **intentIdentifier( )**

  -  **intentPredict()** - Task to complete

*   **check_required_params( )** - Based on the current intents, it goes over it's parameters

*   **check_actions( )** - This function performs the action for the intent

**Note:** Refer the *Chatbot_Reading_Material.pdf* for more information on the conversation flow


       


In [4]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        '''Initialise a default session'''
        # Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts

        # Contexts are flags which control dialogue flow
        self.context = FirstGreeting()

        # Intent tracks the current state of dialogue
        self.current_intent = None

        # attributes hold the information collected over the conversation
        self.attributes = {}

    def reply(self, user_input):
        '''Generate response to user input'''
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)

        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)

        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)

        # prompt being None means all parameters satisfied, perform the intent action
        if prompt is None and self.context.name!='IntentComplete':
            prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)

        return prompt, self.attributes

Created .dat files of slots and Intent in the respective folders. Also updated configuration file in the params folder and CSV file.

The path details of the respective configuration, utterances of the zodiac sign intent and the slots (year, month, day) dat files are provided below,

In [5]:
path_param = 'Chatbot/params/params.cfg'
path_utterances = 'Chatbot/utterances/'
path_slots = 'Chatbot/slots/'

The CSV file path which contains the possible combinations to identify the Zodiac_Sign based on the given date of birth was given

In [6]:
path_csv_zodiac = 'Chatbot/books.csv'

`intentPredict()` function call is specified in the Conversation Flow, which returns the intent to be called in our case it is Zodiac Sign

**Note:** As this pre-hackathon dialogue flow is limited to a single intent, the intentPredict() function is hardcoded to return only the "get Zodiac Sign" intent.

In [7]:
# Take the user input as test data and predict using the model.

def intentPredict(user_input):  # Do not change the function name
    return "get_suggest_book" # Single Intent for a Pre-Hackathon

In [8]:
def get_books_list(df):
  books = []
  for _, row in df.iterrows():
    book = {
        'Title': row['Title'],
        'Author': row['Author'].lower(),
        'Language': row['Language'].lower(),
        'Genre': row['Genre'].lower()
      }
    books.append(book)
  return books

In [9]:
def recommend_book(preferences, books):
  filtered_books = [book for book in books if all(book[key] == value for key, value in preferences.items())]

  if filtered_books:
    return random.choice(filtered_books)
  else:
    return "No matching books found."

In [ ]:
#df = pd.read_csv(path_csv_zodiac)

In [ ]:
#df.head()

In [ ]:
#books_list = get_books_list(df)

In [ ]:
#books_list

Run This API Blocks to perform action after satisfying all the attributes specified for a particular Intent

In [10]:
# Note: Zodiac_sign.csv records are taken from the internet; however it is open to adding multiple records.

# Performs action for zodiac sign with csv file as source
def suggestBook_Action():
    # global session
    attr = session.attributes

    if attr["author"]:
      author = attr["author"].lower()
    else:
      author = None

    if attr["language"]:
      language = attr["language"].lower()
    else:
      language = None

    if attr["genre"]:
      genre = attr["genre"].lower()
    else:
      genre = None

    user_preferences = {}
    if author and author != 'author':
      user_preferences["Author"] = author
    if language and language != 'language':
      user_preferences["Language"] = language
    if genre and genre != 'genre':
      user_preferences["Genre"] = genre

    df = pd.read_csv(path_csv_zodiac)
    books_list = get_books_list(df)
    recommended_book = recommend_book(user_preferences,books_list)
    zodiac = ""

    try:
      if isinstance(recommended_book, dict):
        author_name = recommended_book['Author'].title()
        return f"Recommended Book: {recommended_book['Title']} by {author_name}"
      else:
        return "author is  {author}, language is {language}, genre is {genre}, No matching books found.".format(author=author,language=language,genre=genre)
    except ValueError:
        return "This is not a valid date"

###Main Block to access ChatBot
enter 'end' to stop the bot

Chatbot configuration class

In [11]:
class BOT_config():
    def __init__(self, session):
        self.welcome='BOT: Hi! Welcome to Talentsprint Hackathon, How may i assist you?'
        self.exits=["finish","exit","end","quit","stop","close", "Bye"]
        if session.context.name == 'IntentComplete':
            session.attributes = {}
            session.context = FirstGreeting()
            session.current_intent = None

#### Conversational Chatbot

Interact with the bot by giving any utterance

Ex:  `find zodiac sign`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
session = Session()
print(BOT_config(session).welcome)
while (True):
    inp = input('User: ')
    if inp in BOT_config(session).exits:
        break
    prompt = session.reply(inp)
    print ('BOT:', prompt)

BOT: Hi! Welcome to Talentsprint Hackathon, How may i assist you?
User: suggest book based on Harper Lee English Fiction
BOT: ('Recommended Book: To Kill a Mockingbird by Harper Lee', {'language': 'English', 'genre': 'Fiction', 'author': 'Harper Lee'})
User: suggest book based on George Orwell English Dystopian
BOT: ('Recommended Book: 1984 by George Orwell', {'language': 'English', 'genre': 'Dystopian', 'author': 'George Orwell'})
User: end
